In [1]:
import pandas as pd
import numpy as np
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn.tree
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.cm as cm
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
%matplotlib qt

In [2]:
df = pd.read_csv('C:/Users/XuebinLi/OneDrive - Linden Shore LLC/Desktop/soccer_machine_learning/premier_stats.csv')

In [3]:
index_to_predict = '2nd_half_goals'

In [4]:
# VIF dataframe
def vifdata(X):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns

    # calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                              for i in range(len(X.columns))]

    return vif_data

In [5]:
def importance_random_forest(X_train,y_train):
    from sklearn.ensemble import RandomForestRegressor

    # Create and fit a Random Forest model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Get feature importances
    importances = model.feature_importances_

    # Print feature importances
    for feature, importance in zip(X_train.columns, importances):
        print(f"{feature}: {importance}")

In [6]:
def correlation_matrix(df):   
    plt.figure(figsize = (6, 6))
    heatmap = sns.heatmap(df.corr(), vmin = -1, vmax = 1, annot = True)
    heatmap.set_title('Macro Correlation Heatmap after selection', fontdict = {'fontsize' : 18}, pad = 12)
    return heatmap

In [7]:
print(df.columns)

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'Home Team Pre-Match xG',
       'Away Team Pre-Match xG', 'average_goals_per_match_pre_match',
       'btts_percentage_pre_match', 'over_15_percentage_pre_match',
       'over_25_percentage_pre_match', 'over_35_percentage_pre_match',
       'over_45_percentage_pre_match', 'over_15_HT_FHG_percentage_pre_match',
       'over_05_HT_FHG_percentage_pre_match',
       'over_15_2HG_percentage_pre_match', 'over_05_2HG_percentage_pre_match',
       'average_corners_per_match_pre_match',
       'average_cards_per_match_pre_match', 'odds_ft_home_team_win',
       'odds_ft_draw', 'odds_ft_away_team_win', 'odds_ft_over15',

In [8]:
df['2nd_half_goals'] = df['total_goal_count'] - (df['away_team_goal_count_half_time']+df['home_team_goal_count_half_time'])

In [9]:
df['2nd_half_goals'] = df['2nd_half_goals'].apply(lambda x: 1 if x>=2 else 0)

In [10]:
# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (string)
    if df[column].dtype == 'object':
        # Fit and transform the column
        df[column] = encoder.fit_transform(df[column])

In [11]:
X = df.iloc[:, :-2]
print(X.columns)
Initial_X = X
vifdata(X)

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'Home Team Pre-Match xG',
       'Away Team Pre-Match xG', 'average_goals_per_match_pre_match',
       'btts_percentage_pre_match', 'over_15_percentage_pre_match',
       'over_25_percentage_pre_match', 'over_35_percentage_pre_match',
       'over_45_percentage_pre_match', 'over_15_HT_FHG_percentage_pre_match',
       'over_05_HT_FHG_percentage_pre_match',
       'over_15_2HG_percentage_pre_match', 'over_05_2HG_percentage_pre_match',
       'average_corners_per_match_pre_match',
       'average_cards_per_match_pre_match', 'odds_ft_home_team_win',
       'odds_ft_draw', 'odds_ft_away_team_win', 'odds_ft_over15',

,feature,VIF
0,timestamp,31368.221261
1,date_GMT,7.094008
2,status,NaN
3,attendance,14.677778
4,home_team_name,5.193127
5,away_team_name,5.331489
6,referee,4.132845
7,Game Week,9.981808
8,Pre-Match PPG (Home),18.732954
9,Pre-Match PPG (Away),17.002436


In [12]:
# the independent variables set
VIF = vifdata(X)
features_list_old = VIF['feature'].tolist()
features_list_old.append(index_to_predict)
while VIF['VIF'].max() >= 10:
    max_VIF = VIF['VIF'].idxmax()
    feature_row = VIF.loc[max_VIF,'feature']
    X = X.drop([feature_row],axis=1)
    VIF = vifdata(X)
features_list = VIF['feature'].tolist()
features_list.append(index_to_predict)
print(features_list)

['date_GMT', 'status', 'attendance', 'home_team_name', 'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Away)', 'home_team_goal_count_half_time', 'away_team_goal_count_half_time', 'home_team_first_half_cards', 'home_team_second_half_cards', 'away_team_first_half_cards', 'over_45_percentage_pre_match', 'over_15_HT_FHG_percentage_pre_match', 'odds_ft_home_team_win', 'odds_ft_away_team_win', 'odds_ft_over45', 'stadium_name', '2nd_half_goals']


In [13]:
df = df[features_list]
print(df.columns)

Index(['date_GMT', 'status', 'attendance', 'home_team_name', 'away_team_name',
       'referee', 'Game Week', 'Pre-Match PPG (Away)',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'over_45_percentage_pre_match',
       'over_15_HT_FHG_percentage_pre_match', 'odds_ft_home_team_win',
       'odds_ft_away_team_win', 'odds_ft_over45', 'stadium_name',
       '2nd_half_goals'],
      dtype='object')


In [14]:
print(df['2nd_half_goals'])

0      1
1      0
2      0
3      0
4      0
      ..
375    0
376    0
377    0
378    1
379    1
Name: 2nd_half_goals, Length: 380, dtype: int64


In [15]:
features = df.drop(columns = [index_to_predict], axis = 1)
predict = df[[index_to_predict]]

In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, predict, test_size=0.2)

# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Create a confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
labels = ['Class 0', 'Class 1']
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)

plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [17]:
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0 1 0 1 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0
 0 1]


In [18]:
print(X_train)

     date_GMT  status  attendance  home_team_name  away_team_name  referee  \
131        49       0       25098               6               3       10   
248        88       0       54483              12               1       11   
266       100       0       31778              10               6        2   
273       104       0       59618               1               0        3   
367       152       0       24263               9              13        9   
..        ...     ...         ...             ...             ...      ...   
255        92       0       31438              15               4       11   
299       139       0       10625               0              14       13   
121       169       0       39139               7               4       14   
14         38       0       56888              18               0       17   
106       159       0       31185              19              16       13   

     Game Week  Pre-Match PPG (Away)  home_team_goal_count_half

In [19]:
# logistic regression for feature importance
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from matplotlib import pyplot
# define the model
model = LogisticRegression()
# fit the model
model.fit(X_train, y_train)
# get importance
importance = model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
     print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()
Precision = precision_score(y_test, y_pred)

Feature: 0, Score: 0.00376
Feature: 1, Score: 0.00000
Feature: 2, Score: 0.00000
Feature: 3, Score: -0.00152
Feature: 4, Score: 0.00161
Feature: 5, Score: -0.00857
Feature: 6, Score: 0.01262
Feature: 7, Score: 0.01671
Feature: 8, Score: -0.01235
Feature: 9, Score: -0.00239
Feature: 10, Score: 0.01941
Feature: 11, Score: -0.00517
Feature: 12, Score: -0.01856
Feature: 13, Score: -0.00225
Feature: 14, Score: -0.00478
Feature: 15, Score: 0.00454
Feature: 16, Score: -0.00144
Feature: 17, Score: -0.07340
Feature: 18, Score: -0.02672
